In [ ]:
import uproot
import polars as pl
print("uproot version: ", uproot.__version__)

import sys
import os

import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.file_locations import data_files_location, intermediate_files_location

from src.systematics import get_rw_sys_weights_dic


In [ ]:
num_events = 1000


In [ ]:
all_event_weights = get_rw_sys_weights_dic(
    data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root",
    max_entries=num_events,
)

print(f"len(all_event_weights): {len(all_event_weights)}")
for k, v in all_event_weights[0].items():
    print(f"{k}: {len(v)} weights, first 5: {v[:5]}")

In [ ]:
print(1/0)

In [ ]:
presel_weights_df = pl.read_parquet(intermediate_files_location + "/presel_weights_df.parquet")
presel_weights_df

In [ ]:

f = uproot.open(data_files_location + "/checkout_MCC9.10_Run4a4c4d5_v10_04_07_13_BNB_nu_overlay_surprise_reco2_hist_4c.root")

weight_spline_times_tune = f["nuselection"]["NeutrinoSelectionFilter"]["weightSplineTimesTune"].array(library="np", entry_stop=num_events)
weight_cv = f["wcpselection"]["T_eval"]["weight_cv"].array(library="np", entry_stop=num_events)
weight_spline = f["wcpselection"]["T_eval"]["weight_spline"].array(library="np", entry_stop=num_events)
weight_cv_times_spline = weight_cv * weight_spline

wc_kine_reco_Enus = f["wcpselection"]["T_KINEvars"]["kine_reco_Enu"].array(library="np", entry_stop=num_events)


In [ ]:
weights_All_UBGenie = np.array([event["All_UBGenie"] for event in all_event_weights])
weights_AxFFCCQEshape = np.array([event["AxFFCCQEshape_UBGenie"] for event in all_event_weights])
weights_DecayAngMEC = np.array([event["DecayAngMEC_UBGenie"] for event in all_event_weights])
weights_NormCCCOH = np.array([event["NormCCCOH_UBGenie"] for event in all_event_weights])
weights_NormNCCOH = np.array([event["NormNCCOH_UBGenie"] for event in all_event_weights])
weights_RPA_CCQE = np.array([event["RPA_CCQE_UBGenie"] for event in all_event_weights])
weights_ThetaDelta2NRad = np.array([event["ThetaDelta2NRad_UBGenie"] for event in all_event_weights])
weights_Theta_Delta2Npi = np.array([event["Theta_Delta2Npi_UBGenie"] for event in all_event_weights])
weights_VecFFCCQEshape = np.array([event["VecFFCCQEshape_UBGenie"] for event in all_event_weights])
weights_XSecShape_CCMEC = np.array([event["XSecShape_CCMEC_UBGenie"] for event in all_event_weights])
weights_flux_all = np.array([event["flux_all"] for event in all_event_weights])
weights_reint_all = np.array([event["reint_all"] for event in all_event_weights])
weights_xsr_scc_Fa3_SCC = np.array([event["xsr_scc_Fa3_SCC"] for event in all_event_weights])
weights_xsr_scc_Fv3_SCC = np.array([event["xsr_scc_Fv3_SCC"] for event in all_event_weights])


In [ ]:
bins = np.linspace(0, 2000, 11)

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_All_UBGenie.shape[1])):
    curr_event_weights = weights_All_UBGenie[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"GENIE_All uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_AxFFCCQEshape.shape[1])):
    curr_event_weights = weights_AxFFCCQEshape[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"AxFFCCQEshape uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_DecayAngMEC.shape[1])):
    curr_event_weights = weights_DecayAngMEC[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"DecayAngMEC uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_NormCCCOH.shape[1])):
    curr_event_weights = weights_NormCCCOH[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"NormCCCOH uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_NormNCCOH.shape[1])):
    curr_event_weights = weights_NormNCCOH[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"NormNCCOH uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_RPA_CCQE.shape[1])):
    curr_event_weights = weights_RPA_CCQE[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"RPA_CCQE uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_ThetaDelta2NRad.shape[1])):
    curr_event_weights = weights_ThetaDelta2NRad[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"ThetaDelta2NRad uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_Theta_Delta2Npi.shape[1])):
    curr_event_weights = weights_Theta_Delta2Npi[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"Theta_Delta2Npi uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_VecFFCCQEshape.shape[1])):
    curr_event_weights = weights_VecFFCCQEshape[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"VecFFCCQEshape uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_XSecShape_CCMEC.shape[1])):
    curr_event_weights = weights_XSecShape_CCMEC[:, uni_i] * weight_spline
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"XSecShape_CCMEC uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_flux_all.shape[1])):
    curr_event_weights = weights_flux_all[:, uni_i] * weight_spline_times_tune
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"flux_all uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_reint_all.shape[1])):
    curr_event_weights = weights_reint_all[:, uni_i] * weight_spline_times_tune
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"reint_all uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_xsr_scc_Fa3_SCC.shape[1])):
    curr_event_weights = weights_xsr_scc_Fa3_SCC[:, uni_i] * weight_spline_times_tune # for the xsr_scc weights, need to multiply by the CV weight
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"xsr_scc_Fa3_SCC uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()

plt.figure()
plt.hist(wc_kine_reco_Enus, weights=weight_spline_times_tune, bins=bins, histtype="step", label="CV", color="k", lw=2)
for uni_i in range(min(10, weights_xsr_scc_Fv3_SCC.shape[1])):
    curr_event_weights = weights_xsr_scc_Fv3_SCC[:, uni_i] * weight_spline_times_tune # for the xsr_scc weights, need to multiply by the CV weight
    plt.hist(wc_kine_reco_Enus, weights=curr_event_weights, bins=bins, histtype="step", label=f"xsr_scc_Fv3_SCC uni {uni_i}")
plt.xlabel("wc_kine_reco_Enu")
plt.ylabel("weighted event count")
plt.legend()
plt.show()
